In [331]:
import json
import pytz
import pandas as pd
import numpy as np
import plotly.express as px

from plotly.subplots import make_subplots
from datetime import datetime
from time import mktime
from dateutil.relativedelta import relativedelta

def getyearmonth(row):
    return datetime.utcfromtimestamp(row['payment_time.$date']/1000).strftime('%Y-%m')

f = open('temp/test.json')
myjson = json.load(f)

orders = pd.json_normalize(myjson)
orders['myitems'] = orders.apply(lambda row: row["items"] if row["items"]==row["items"] else row["group.my_member.items"], axis=1)
orders['myprices'] = orders.apply(lambda row: row["total_price_share"] if row["total_price_share"] > 0 else row["total_price"], axis=1)
orders['venue_name_fixed'] = orders['venue_name'].str.extract(r'([^\\/|+]*)')
orders["year-month"] = orders.apply(lambda row: getyearmonth(row), axis=1)

lastYear = mktime((datetime.today().date().replace(day=1) - relativedelta(years=1)).timetuple()) * 1000
orders = orders[orders['delivery_time.$date'] >= lastYear]

In [332]:
monthly = (orders.groupby(['currency', 'year-month']).sum('total_price').total_price/100).reset_index(level=['currency', 'year-month'])

In [333]:
items = orders.explode('myitems')
items["item_name"] = items['myitems'].map(lambda x : x["name"]) 
items["item_count"] = items['myitems'].map(lambda x : x["count"])
items["item_total_price"] = items['myitems'].map(lambda x : x["end_amount"])
len(items.index)


192

In [334]:
orders['order_id'].count()

64

In [335]:
totals = (orders[['currency','myprices']].groupby(['currency']).sum().sort_values('myprices', ascending=False)/100).to_dict()['myprices']
" / ".join(f'{tup[1]:.2f} {tup[0]}'  for tup in list(totals.items()))

'8538.00 ILS / 2332.00 USD / 671.00 EUR'

In [336]:
averages = (orders[['currency','myprices']].groupby(['currency']).mean().sort_values('myprices', ascending=False)/100).to_dict()['myprices']
" / ".join(f'{tup[1]:.2f} {tup[0]}'  for tup in list(averages.items()))

'181.66 ILS / 179.38 USD / 167.75 EUR'

In [337]:
orders_heatmap = pd.DataFrame(np.zeros((5, 7)))

for index, row in orders.iterrows():

    timezone = pytz.timezone(row["venue_timezone"])
    timestamp = datetime.fromtimestamp(
        row['payment_time.$date']/1000, timezone)

    weekday = int(timestamp.strftime('%w'))
    hour = int(timestamp.strftime('%H'))

    if hour <= 6:
        timeofday = 4
    elif hour <= 12:
        timeofday = 0
    elif hour <= 16:
        timeofday = 1
    elif hour <= 19:
        timeofday = 2
    elif hour <= 24:
        timeofday = 3
    else:
        timeofday = 4

    orders_heatmap[weekday][timeofday] += 1

labels_x = ['Sunday', 'Monday', 'Tuesday',
            'Wednesday', 'Thursday', 'Friday', 'Saturday']
labels_y = ['Morning (6-12)', 'Noon (12-16)',
            'Afternoon (16-19)', 'Evening (19-22)', 'Night (22-6)']

fig = px.imshow(orders_heatmap, y=labels_y, x=labels_x,
                aspect='auto', text_auto=True, height=600, width=600)
fig.update_layout(coloraxis_showscale=False)
fig.update_xaxes(side="top")

fig

In [338]:
everything = items.groupby(['currency', 'venue_name_fixed', 'item_name']).agg(
    {"item_total_price": lambda x: np.sum(x)/100}).reset_index()

fig = px.treemap(everything, path=['currency','venue_name_fixed', 'item_name'], values=('item_total_price'))
fig.update_traces(texttemplate='%{label} = %{value}')
fig.update_layout(margin=dict(t=10, l=10, r=10, b=10))


In [339]:
chart = px.bar(monthly, title="Monthly Expenses", text_auto=True, barmode='group', color='currency', x='year-month', y='total_price',
               labels={"value": "Total Expense", "year-month": "Month"})
chart.update_layout(showlegend=False)
chart.update_xaxes(tickangle=30)